The second project with pretrained model to train for the dogs

In [35]:
#import necessary packages
import sys
sys.path.insert(0, "../../")

import collections
import datetime
import gluonbook as gb
import math
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import data as gdata, loss as gloss, model_zoo, nn
import os
import shutil
import zipfile

In [36]:
#extractall the files
demo=False
data_dir = "../../data/kaggle_dog/"

if demo:
    zipfiles = ["train_valid_test_tiny.zip"]
else:
    zipfiles = ["test.zip","train.zip", "labels.csv.zip"]
if not os.path.exists(data_dir+"train") or demo:
    for f in zipfiles:
        with zipfile.ZipFile(data_dir+f, 'r') as z:
            z.extractall(data_dir)

In [37]:
#orgnize the data for the later load
def regor_dog_data(data_dir, train_data, test_data, label_data, input_dir, valid_rate):
    # open the label file and find out the least train dogs sample number
    with open(data_dir+label_data, 'r') as f:
        lines=f.readlines()[1:]
        tokens = [l.rstrip().split(",") for l in lines]
        idx_label=dict((idx, label) for idx, label in tokens)
    
    
    def mkdir_if_not_exist(path):
        if not os.path.exists(os.path.join(*path)):
            os.makedirs(os.path.join(*path))
        
    minmum_dog_number = (collections.Counter(idx_label.values()).most_common()[:-2:-1][0][1])
    #mkdir for each label in train and valid folder, copy the files into each label folder
    # put the test data into input test folder with unknow label
    valid_sample_number = math.floor(minmum_dog_number*valid_rate)
    copy_dict={}
    
    
    for train_file in os.listdir(os.path.join(data_dir, train_data)):
        idx = train_file.split(".")[0]
        label =idx_label[idx]
        mkdir_if_not_exist([data_dir, input_dir, "train_valid", label])
        shutil.copy(os.path.join(data_dir, train_data,train_file), os.path.join(data_dir, input_dir,"train_valid", label))
        if label not in copy_dict or copy_dict[label] < valid_sample_number:
            mkdir_if_not_exist([data_dir, input_dir,"valid", label])
            shutil.copy(os.path.join(data_dir, train_data,train_file), os.path.join(data_dir, input_dir,"valid", label))
            copy_dict[label] = copy_dict.get(label,0)+1
            #if I use copy_dict[label]+=1, get bedlington_terrier
        else:
            mkdir_if_not_exist([data_dir, input_dir,"train", label])
            shutil.copy(os.path.join(data_dir, train_data,train_file), os.path.join(data_dir, input_dir,"train", label))
    for test_file in os.listdir(os.path.join(data_dir, test_data)):
        mkdir_if_not_exist([data_dir, input_dir, "test", "unknow"])
        shutil.copy(os.path.join(data_dir, test_data,test_file), os.path.join(data_dir, input_dir,"test", "unknow"))

In [38]:
if demo:
    input_dir, batch_size = "train_valid_test_tiny", 1
else:
    input_dir, train_data, test_data, label_file= "input_dir", "train","test","labels.csv"
    batch_size,valid_rate =128, 0.1
    regor_dog_data(data_dir,train_data,test_data,label_file,input_dir,0.1)

In [39]:
transform_train = gdata.vision.transforms.Compose([
    # 随机对图像裁剪出面积为原图像面积 0.08 到 1 倍之间、且高和宽之比在 3/4 和 4/3 之间
    # 的图像，再放缩为高和宽均为 224 像素的新图像。
    gdata.vision.transforms.RandomResizedCrop(224, scale=(0.08, 1.0),
                                              ratio=(3.0/4.0, 4.0/3.0)),
    # 随机左右翻转图像。
    gdata.vision.transforms.RandomFlipLeftRight(),
    # 随机抖动亮度、对比度和饱和度。
    gdata.vision.transforms.RandomColorJitter(brightness=0.4, contrast=0.4,
                                              saturation=0.4),
    # 随机加噪音。
    gdata.vision.transforms.RandomLighting(0.1),
    
    # 将图像像素值按比例缩小到 0 和 1 之间，并将数据格式从“高 * 宽 * 通道”改为
    # “通道 * 高 * 宽”。
    gdata.vision.transforms.ToTensor(),
    # 对图像的每个通道做标准化。
    gdata.vision.transforms.Normalize([0.485, 0.456, 0.406],
                                      [0.229, 0.224, 0.225])])

# 测试时，只使用确定性的图像预处理操作。
transform_test = gdata.vision.transforms.Compose([
    gdata.vision.transforms.Resize(256),
    # 将图像中央的高和宽均为 224 的正方形区域裁剪出来。
    gdata.vision.transforms.CenterCrop(224),
    gdata.vision.transforms.ToTensor(),
    gdata.vision.transforms.Normalize([0.485, 0.456, 0.406],
                                      [0.229, 0.224, 0.225])])

In [40]:
train_ds = gdata.vision.ImageFolderDataset(os.path.join(data_dir,input_dir,"train"), flag=1)
valid_ds = gdata.vision.ImageFolderDataset(os.path.join(data_dir,input_dir,"valid"), flag=1)
train_valid_ds = gdata.vision.ImageFolderDataset(os.path.join(data_dir,input_dir,"train_valid"), flag=1)
test_ds = gdata.vision.ImageFolderDataset(os.path.join(data_dir,input_dir,"test"), flag=1)

train_data=gdata.DataLoader(train_ds.transform_first(transform_train), batch_size, shuffle=True, last_batch='keep')
valid_data=gdata.DataLoader(valid_ds.transform_first(transform_train), batch_size, shuffle=True, last_batch='keep')
train_valid_data=gdata.DataLoader(train_valid_ds.transform_first(transform_train), batch_size, shuffle=True, last_batch='keep')
test_data=gdata.DataLoader(test_ds.transform_first(transform_test), batch_size, shuffle=False, last_batch='keep')

In [41]:
def get_net(ctx):
    finetune_net= model_zoo.vision.resnet34_v2(pretrained=True)
    finetune_net.output_new = nn.HybridSequential(prefix="")
    finetune_net.output_new.add(nn.Dense(256, activation='relu'))
    finetune_net.output_new.add(nn.Dense(120))
    finetune_net.output_new.initialize(init.Xavier(), ctx=ctx)
    # need to put the parameters on to model cpu or gpu()
    finetune_net.collect_params().reset_ctx(ctx)
    return finetune_net

In [42]:
#If sparse_label is True (default), label should contain integer category indicators:
loss = gloss.SoftmaxCrossEntropyLoss()

# Use the get_loss to get the loss for the valid data, can not use the net(X) directly because we will not use the
# old output layer
def get_loss(data, net, ctx):
    l=0
    for X, y in data:
        features_out = net.features(X.as_in_context(ctx))
        output=net.output_new(features_out)
        l += loss(output, y.as_in_context(ctx)).mean().asscalar()
    return l/len(data)

In [43]:
def train(net, train_data, valid_data, lr, wd, num_epochs, lr_decay,lr_period, batch_size, ctx):
    trainer = gluon.Trainer(net.output_new.collect_params(),'sgd', {'learning_rate':lr, 'momentum':0.9, 'wd': wd})
    previous_time=datetime.datetime.now()
    for epoch in range(num_epochs):
        train_l = 0
        if epoch > 0 and epoch%lr_period == 0:
            trainer.set_learning_rate(trainer.learning_rate*lr_decay)
        for X, y in train_data:
            features_out = net.features(X.as_in_context(ctx))
            with autograd.record():
                output = net.output_new(features_out)
                #l = loss(output,y.astype('float32').as_in_context(ctx))
                l = loss(output,y.as_in_context(ctx))

            l.backward()
            #print(l)
            train_l += l.mean().asscalar()
            trainer.step(batch_size)
            
        current_time=datetime.datetime.now()
        
        h,reminder=divmod((current_time-previous_time).seconds, 3600)
        m,s = divmod(reminder, 60)
        time_s = "time: %02d:%02d:%02d" %(h,m,s)
        
        #print(train_l)
        if epoch % 1 == 0:
            if valid_data is not None:  
                print("epoch %d, train_loss %f, valid_loss %f, learning_rate %f, time %s"%
                      (epoch, train_l/len(train_data), get_loss(valid_data, net, ctx), 
                      trainer.learning_rate, time_s))
            else:
                print("epoch %d, train_loss %f, learning_rate %f, time %s"%
                      (epoch, train_l/len(train_data),trainer.learning_rate,time_s))

In [ ]:
ctx, num_epochs, lr, wd = gb.try_gpu(), 100, 0.0001, 1e-4
lr_period, lr_decay, net = 10, 0.1, get_net(ctx)
net.hybridize()
net.load_parameters(os.path.join("../../data/", "dog_2019_09_02_16_14.params"))
train(net, train_data, valid_data, lr, wd, num_epochs, 
      lr_decay, lr_period, batch_size, ctx)

epoch 0, train_loss 0.708829, valid_loss 0.969698, learning_rate 0.000100, time time: 00:01:18
epoch 1, train_loss 0.691222, valid_loss 0.847319, learning_rate 0.000100, time time: 00:02:42
epoch 2, train_loss 0.685034, valid_loss 0.840691, learning_rate 0.000100, time time: 00:04:06
epoch 3, train_loss 0.704227, valid_loss 0.820492, learning_rate 0.000100, time time: 00:05:31
epoch 4, train_loss 0.704574, valid_loss 0.763563, learning_rate 0.000100, time time: 00:06:55
epoch 5, train_loss 0.704703, valid_loss 0.892400, learning_rate 0.000100, time time: 00:08:19
epoch 6, train_loss 0.663265, valid_loss 0.865297, learning_rate 0.000100, time time: 00:09:44
epoch 7, train_loss 0.707593, valid_loss 0.918001, learning_rate 0.000100, time time: 00:11:08
epoch 8, train_loss 0.676576, valid_loss 0.792126, learning_rate 0.000100, time time: 00:12:32
epoch 9, train_loss 0.701074, valid_loss 0.829652, learning_rate 0.000100, time time: 00:13:56
epoch 10, train_loss 0.661341, valid_loss 0.896218

In [ ]:
filename = os.path.join("../../data/", "dog_2019_09_02_night.params")
net.save_parameters(filename)